In [1]:
# All packages to import
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import ensemble

from feature_engine import encoding as ce
from feature_engine import transformation as tran
from feature_engine import outliers as out
from feature_engine import selection as select

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing as prep
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import LassoLars
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import PoissonRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.decomposition import PCA


from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from regressors import stats
import math
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.DataFrame()
startPath = 'C://Users//asd25/'
bpg = ['AIR FILTERS','BATTERIES','EXHAUST','GASKETS','MOTOR OIL','WIPERS']
table = 'bottoms_up_gt'
for b in bpg:
    dff = pd.read_csv(startPath+b+' '+table+'.csv')
    #s= dff.sample(frac=0.3) 
    df = pd.concat([df,dff])
    print(b, df.shape)

AIR FILTERS (151746, 55)
BATTERIES (389928, 55)
EXHAUST (398057, 55)
GASKETS (486837, 55)
MOTOR OIL (1162184, 55)
WIPERS (1567803, 55)


In [3]:
#stratified - 2
df=df.sample(frac=0.2, weights='qty_sold_cy',random_state=1).reset_index(drop=True)

In [4]:
changeBU = {'sku_store_pdq':'object',
             'store_number':'object','sku_number':'object',
             'part_type':'object','mpog_id':'object',}
df = df.astype(changeBU)

In [5]:
# Remove columns with "cy"
'''If you are using bottoms_up, make remove = []'''
def RemoveCY(df,keep=["qty_sold_cy"]): # keep is the variable that have cy, but you want to keep
    remove=[] # cat is also a cy variable
    for col in df.columns:
        if (col.find('_cy')>=0 or col.find('cy_')>=0) and col not in keep:
            remove.append(col)
    return df.drop(columns=remove)
# prep.FunctionTransformer(RemoveCY,kw_args={"keep":keep})

#Remove non-unique columns
def dropSingles(df):
    drops = []
    for col in df.columns:
        if len(df[col].unique())==1:
            drops.append(col)
    df = df.drop(columns=drops)
    return df
# prep.FunctionTransformer(dropSingles)

df = RemoveCY(dropSingles(df))

In [16]:
target = df['qty_sold_cy']
predictors = df.drop(columns = ['qty_sold_cy','qty_sold_py','sku_store_pdq', 'qty_sold', 'filter_reason', 'platform_cluster_name', 'part_type','avg_cluster_total_sales'])
X_train, X_test, y_train, y_test = train_test_split(
    predictors, # predictors
    target,  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

In [17]:
categories = []
discrete = []
PosContinuous = []
NegContinuous = ["unit_sales", "other_unit_pls_lost_sales", 'adjusted_avg_cluster_sales',
                "other_unit_pls_lost_sales_py", "avg_cluster_unit_sales", 
                "ntrans_wt0_py", "ntrans_wt0_ppy", "weighted_lookup_cnt",
                "avg_cluster_total_sales", "adj_avg_cluster_total_sales"] # These will change
continuous = []
for col in X_train.columns:
    if col in NegContinuous:
        continuous.append(col)
    elif X_train[col].dtype.name =="object":
        categories.append(col)
    elif X_train[col].dtype.name == "int64":
        discrete.append(col)
    else:
        PosContinuous.append(col)
        continuous.append(col)

In [18]:
# Remove high correlation
def highCorr(df,keep,cutOff):
    '''df is the dataframe
        keep is a list of variables to not include in removing correlation
            probably a single target variable, but still put in list
        cutOff is the correlation cut off to remove'''
    df = df.drop(columns=keep)
    corr = df.corr()
    variables = corr.columns
    correlated_features = set()
    for r in range(len(variables)):
        for c in range(r):
            if abs(corr.iloc[r,c])>cutOff:
                colname = variables[r]
                correlated_features.add(colname)
    return df.drop(columns = correlated_features)
# prep.FunctionTransformer(highCorr,kw_args={"keep":keep,"cutOff":0.95})

# Less than zero
def MakePos(df,negs = None): #negs is the list of numeric columns to retain negative values
    for col in df. columns:
        if df[col].dtype.name =="category" or df[col].dtype.name =="object":
            continue
        if col in negs:
            continue
        if np.min(df[col])<0:
            d = df[col].copy()
            d = np.where(d < 0,0,d)
            df[col]=d
    return df
# prep.FunctionTransformer(MakePos, kw_args={"negs": NegContinuous})


In [19]:
def log_transform(X,variables): #Works, only pass variables >=0
    result = X.copy()
    for col in result.columns:
        if col in variables:
            result[col] = np.log(result[col]+1)
    return result
# prep.FunctionTransformer(log_transform, kw_args={"variables":var})

def ratios(X, variables, tuples = False): # Works
    '''the variables can either be a list of variables or a list of
    pairs of variables (numerator,denominator): set tuples = True.
    If tuples=False, then every variable in the list will be a numerator
    or denominator with every other variable. tuples=True is for when
    you have specific pairings you want to use.'''
    result = X.copy()
    if tuples:
        for n,d in variables:
            denom = np.where(result[d]==0,0.001,result[d])
            result[n+"/"+d] = result[n]/denom
    else:
        for d in variables:
            denom = np.where(X[d]==0,0.001,X[d])
            for n in variables:
                if n!=d:
                    result[n+"/"+d] = result[n]/denom
    return result
# prep.FunctionTransformer(ratios, kw_args={"variables" : var, "tuples" : False})

In [20]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
def MAPE(y_true,y_pred):
    y_true = np.where(y_true==0,0.001,y_true)
    return mean_absolute_percentage_error(y_true,y_pred)
def SMAPE(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [11]:
#naive forecast

y_actual= df.qty_sold_cy # replace with y_train or y_test as needed
y_prediction = df.qty_sold_py # replace with y_pred_tr or y_pred_te as needed

print("RMSE:",math.sqrt(mean_squared_error(y_actual,y_prediction)))
#print("MAPE:",mean_absolute_percentage_error(y_actual,y_prediction))
print("MAE:",mean_absolute_error(y_actual, y_prediction))
print("R^2:",r2_score(y_actual, y_prediction))

RMSE: 215.77513124837398
MAE: 52.859421292826596
R^2: 0.32030268127507444


In [21]:
pipe=Pipeline([("makePos", prep.FunctionTransformer(MakePos, kw_args={"negs": NegContinuous})),
               ("rare", ce.RareLabelEncoder(tol=0.01,n_categories=7)),
               ("nzv", select.DropConstantFeatures(tol=0.95)),
               ("filter_corr", select.DropCorrelatedFeatures(threshold=0.95)),])
pipe.fit(X_train,y_train)

X_train = pipe.transform(X_train)
X_test = pipe.transform(X_test)

In [25]:
categories = []
discrete = []
PosContinuous = []
NegContinuous = ["unit_sales", "other_unit_pls_lost_sales", 'adjusted_avg_cluster_sales',
                "other_unit_pls_lost_sales_py", "avg_cluster_unit_sales", 
                "ntrans_wt0_py", "ntrans_wt0_ppy", "weighted_lookup_cnt",
                "avg_cluster_total_sales", "adj_avg_cluster_total_sales"]# These will change

continuous = []
for col in X_train.columns:
    if col in NegContinuous:
        continuous.append(col)
    elif X_train[col].dtype.name =="object":
        categories.append(col)
    elif X_train[col].dtype.name == "int64":
        discrete.append(col)
    else:
        PosContinuous.append(col)
        continuous.append(col)

In [26]:
continuous

['pop_est',
 'pop_density',
 'total_vio',
 'avg_cluster_unit_sales',
 'adjusted_avg_cluster_sales',
 'sales_signal',
 'failure_sales',
 'lifecycle',
 'unit_sales',
 'projected_growth_pct',
 'weighted_lookup_cnt',
 'qty_wt0_ppy',
 'ntrans_wt0_ppy',
 'ntrans_wt0_py',
 'ntrans_wt0',
 'unadjusted_total_vio',
 'vio_compared_to_cluster',
 'qty_wt0',
 'pct_white',
 'age',
 'pct_college',
 'pct_blue_collar',
 'median_household_income',
 'establishments',
 'road_quality_index',
 'pct_of_lifecycle_remaining',
 'trend']

In [30]:
#Sensitivity
# Loop-d-Loop
# Right now, I am doing all categories the same, might change variables later
pipes = {"G1":{"cat_encode":ce.MeanEncoder(),
                "model":linear_model.Ridge()},
        "G2":{"cat_encode":ce.MeanEncoder(),
                "model":LinearRegression()},
        "G3":{"cat_encode":ce.MeanEncoder(),
               "model":BayesianRidge()},
        "G4":{"cat_encode":ce.MeanEncoder(),
                "model":GradientBoostingRegressor(random_state=0)},
       }

perChange=[-.9,-.8,-.7,-.6,-.5,-.4,-.3,-.2,-.1,0,.1,.2,.3,.4,.5,.6,.7,.8,.9]

FEATURE = []
IMPORTANCE=[]
CHANGE = []
MEAN_CHANGE =[]
STD=[]
MAX=[]
MIN=[]
SENSITIVITY = []
ID = []

for i,pipe in pipes.items():
    X_tr = X_train.copy()
    X_te = X_test.copy()
    cat = pipe["cat_encode"].fit(X_tr,y_train)
    X_tr=cat.transform(X_tr)
    X_te_base = cat.transform(X_te)
    names = X_te.columns
    model = pipe["model"].fit(X_tr,y_train)
    baseline = model.predict(X_te_base)
    try:
        importance = model.feature_importances_
        d2 = pd.DataFrame([importance],columns = names)
    except:
        importance = stats.coef_pval(model, X_tr, y_train)
        importance = np.delete(importance, 0)
        d2 = pd.DataFrame([importance],columns = names)
    for col in continuous:
        imp = d2.loc[0,col]
        for p in perChange:
            print(i)
            print(col)
            print(p)
            X_te = X_te_base.copy()
            X_te[col]=X_te[col]*(1+p)
            new = model.predict(X_te)
            change = new-baseline
            perC = change/baseline
            if p ==0:
                sensitivity = perC/0.001
            else:
                sensitivity = perC/p
            sen = sensitivity.mean()
            IMPORTANCE.append(imp)
            FEATURE.append(col)
            ID.append(i)
            CHANGE.append(p)
            SENSITIVITY.append(sen)
            STD.append(np.std(sensitivity))
            MAX.append(sensitivity.max())
            MIN.append(sensitivity.min())

G1
pop_est
-0.9
G1
pop_est
-0.8
G1
pop_est
-0.7
G1
pop_est
-0.6
G1
pop_est
-0.5
G1
pop_est
-0.4
G1
pop_est
-0.3
G1
pop_est
-0.2
G1
pop_est
-0.1
G1
pop_est
0
G1
pop_est
0.1
G1
pop_est
0.2
G1
pop_est
0.3
G1
pop_est
0.4
G1
pop_est
0.5
G1
pop_est
0.6
G1
pop_est
0.7
G1
pop_est
0.8
G1
pop_est
0.9
G1
pop_density
-0.9
G1
pop_density
-0.8
G1
pop_density
-0.7
G1
pop_density
-0.6
G1
pop_density
-0.5
G1
pop_density
-0.4
G1
pop_density
-0.3
G1
pop_density
-0.2
G1
pop_density
-0.1
G1
pop_density
0
G1
pop_density
0.1
G1
pop_density
0.2
G1
pop_density
0.3
G1
pop_density
0.4
G1
pop_density
0.5
G1
pop_density
0.6
G1
pop_density
0.7
G1
pop_density
0.8
G1
pop_density
0.9
G1
total_vio
-0.9
G1
total_vio
-0.8
G1
total_vio
-0.7
G1
total_vio
-0.6
G1
total_vio
-0.5
G1
total_vio
-0.4
G1
total_vio
-0.3
G1
total_vio
-0.2
G1
total_vio
-0.1
G1
total_vio
0
G1
total_vio
0.1
G1
total_vio
0.2
G1
total_vio
0.3
G1
total_vio
0.4
G1
total_vio
0.5
G1
total_vio
0.6
G1
total_vio
0.7
G1
total_vio
0.8
G1
total_vio
0.9
G1
avg_clu

In [31]:
results = pd.DataFrame({"ID":ID,
                        "FEATURE":FEATURE,
                       "IMPORTANCE":IMPORTANCE,
                       "CHANGE":CHANGE,
                        "SENSITIVITY":SENSITIVITY,
                        "STD":STD,
                        "MIN":MIN,
                        "MAX":MAX
                       }) 
results.head(20)

,ID,FEATURE,IMPORTANCE,CHANGE,SENSITIVITY,STD,MIN,MAX
0,G1,pop_est,0.053012,-0.9,0.000054,0.000093,-0.0,0.003910
1,G1,pop_est,0.053012,-0.8,0.000054,0.000093,-0.0,0.003910
2,G1,pop_est,0.053012,-0.7,0.000054,0.000093,-0.0,0.003910
3,G1,pop_est,0.053012,-0.6,0.000054,0.000093,-0.0,0.003910
4,G1,pop_est,0.053012,-0.5,0.000054,0.000093,-0.0,0.003910
5,G1,pop_est,0.053012,-0.4,0.000054,0.000093,-0.0,0.003910
6,G1,pop_est,0.053012,-0.3,0.000054,0.000093,-0.0,0.003910
7,G1,pop_est,0.053012,-0.2,0.000054,0.000093,-0.0,0.003910
8,G1,pop_est,0.053012,-0.1,0.000054,0.000093,-0.0,0.003910
9,G1,pop_est,0.053012,0.0,0.000000,0.000000,0.0,0.000000


In [32]:
results

,ID,FEATURE,IMPORTANCE,CHANGE,SENSITIVITY,STD,MIN,MAX
0,G1,pop_est,0.053012,-0.9,5.427483e-05,0.000093,-0.0,0.003910
1,G1,pop_est,0.053012,-0.8,5.427483e-05,0.000093,-0.0,0.003910
2,G1,pop_est,0.053012,-0.7,5.427483e-05,0.000093,-0.0,0.003910
3,G1,pop_est,0.053012,-0.6,5.427483e-05,0.000093,-0.0,0.003910
4,G1,pop_est,0.053012,-0.5,5.427483e-05,0.000093,-0.0,0.003910
...,...,...,...,...,...,...,...,...
2047,G4,trend,0.000636,0.5,6.876792e-07,0.000140,0.0,0.033592
2048,G4,trend,0.000636,0.6,8.419413e-07,0.000143,0.0,0.027994
2049,G4,trend,0.000636,0.7,7.216640e-07,0.000122,0.0,0.023994
2050,G4,trend,0.000636,0.8,6.314560e-07,0.000107,0.0,0.020995


In [33]:
results.to_csv("Bottom_up_gt_Sensitivity.csv",index=False)